# BM算法定义规则

1. 坏字符规则：当文本串中的某个字符跟模式串中的某个字符不匹配时，我们称文本串中的这个失配字符为坏字符，此时模式串需要向右移动，移动位数 =  坏字符串在模式中的位置 -  坏字符串在模式串中最右出现的位置。此外，如果坏字符串不出现在模式串之中，则最右出现位置为-1 .  
2. 好后缀原则：当字符失配时，后移位数 = 好后缀在模式串中的位置 - 好后缀在模式串中上一次出现的位置 ，且如果好后缀在模式串中没有再次出现，则为 -1 

   下面举例说明BM算法。例如，给定文本串“HERE IS A SIMPLE EXAMPLE”，和模式串“EXAMPLE”，现要查找模式串是否在文本串中，如果存在，返回模式串在文本串中的位置。


1. 首先，文本串与模式串头部对齐，从尾部开始比较，S和E不匹配，这时，S就被称为坏字符，即不匹配的字符，他对应模式串中的第六位，
且S不包含在EXAMPLE中（相当于最右出现位置为-1），这时可以把模式串右移 6-（-1）=7位，从而直接移动到S的后一位 ![1](./BM-1.png)  
2. 依然从尾部开始比较，P和E不匹配，所以P是坏字符，但是P包含在模式串“EXAMPLE”中，因为文本串中的P这个坏字符对应模式串中的第6位（这里也就是E啦，从0开始编号），且在模式串中最右出现的位置为4，所以将模式串右移 6-4=2位，两个P对齐。 ![1](./BM-2.png)  ![1](./BM-3.png) 
3. 从后边一次比较，得到"MPLE"匹配，称为好匹配，即所有尾部匹配的字符串，注意 "MPLE"、"PLE"、"LE"、"E"都是好后缀。![1](./BM-4.png)  
4. 发现“I”与“A”不匹配：“I”是坏字符。如果是根据坏字符规则，此时模式串应该后移2-(-1)=3位。问题是，有没有更优的移法？![1](./BM-5.png) ![1](./BM-6.png)   
5. 更优的移法是利用好后缀规则：当字符失配时，后移位数 = 好后缀在模式串中的位置 - 好后缀在模式串中上一次出现的位置，且如果好后缀在模式串中没有再次出现，则为-1。所有的“好后缀”（MPLE、PLE、LE、E）之中，只有“E”在“EXAMPLE”的头部出现，所以后移6-0=6位。可以看出，“坏字符规则”只能移3位，“好后缀规则”可以移6位。每次后移这两个规则之中的较大值。这两个规则的移动位数，只与模式串有关，与原文本串无关。![1](./BM-7.png)  
6. 继续从尾部开始比较，“P”与“E”不匹配，因此“P”是“坏字符”，根据“坏字符规则”，后移 6 - 4 = 2位。因为是最后一位就失配，尚未获得好后缀。![1](./BM-8.png) 


In [49]:
# 如下开始具体的算法
# 我用logging吧
import logging
# workaround via specifying an invalid value first
#%config Application.log_level='WORKAROUND'
# => fails, necessary on Fedora 27, ipython3 6.2.1
%config Application.log_level='DEBUG'
import logging
logging.getLogger().setLevel(logging.DEBUG)
log = logging.getLogger()
log.debug('Test debug')

DEBUG:root:Test debug


In [50]:
#logging.basicConfig(level=logging.INFO)


In [56]:
# 先看看取得移动的
def get_good_char_move(p:str):
    """这个是取得好字符移动的"""
    # 当有好字符的时候，是从最后算的
    # 比如 " hello world" ,好字符是从尾部开始算的，
    # 比如最后的 "rld" 三个字符是好字符，我要看看"rld”/"ld"/"和d"三个字符串，都是从左边开始比较，找到匹配的位置（假设有），然后这3个位置比较，谁最小，选谁，
    # 我这里用递推的方式来做 ，因为假设如上3个字符是有位置信息的，那么只能以最大的为准
    _char_move=[-1]*len(p) # 默认为-1啦
    logging.info("处理模式串:{0}".format(p))
    logging.info("生成初始数组:{0}".format(_char_move))
    _index = len(p)-1 # 最后一位开始啦
    while _index > 0 : # 应该不用算0位吧
        logging.info("处理模式串的下标{0}".format(_index))
        _s=p[_index:] # 截取字符串。
        logging.info("假设这个子串是好字符:{0}".format(_s))
        # 然后判断这个字符串的前面是否有位置
        # 这里的rfind是从右边查找。因为是上一个出现的位置，所以是从右边开始查找。
        _j =  p[:_index].rfind(_s) # 这里我用python的函数，只是做例子。
        logging.info("这个子串上一次出现的位置是:{0}".format(_j))
        # 判断是否是一个位置吧
        if _j == _index :
            logging.info("这个子串只出现一次，什么都不用做")
            pass # 如果是一个位置，就什么都不用做
        else:
            _char_move[_index] = _j
            logging.info("暂时设置这个下标，现在这个数组如下")
            logging.info(_char_move)
            
        logging.info("因为这个子串的子串或许有更好的位置，现在数组如下")
        _char_move[_index] = max(_char_move[_index:])
        logging.info(_char_move)
        _index = _index-1
        logging.info("********************************************")
    return _char_move
         
    pass

In [57]:
get_good_char_move("abcabcab")

INFO:root:处理模式串:abcabcab
INFO:root:生成初始数组:[-1, -1, -1, -1, -1, -1, -1, -1]
INFO:root:处理模式串的下标7
INFO:root:假设这个子串是好字符:b
INFO:root:这个子串上一次出现的位置是:4
INFO:root:暂时设置这个下标，现在这个数组如下
INFO:root:[-1, -1, -1, -1, -1, -1, -1, 4]
INFO:root:因为这个子串的子串或许有更好的位置，现在数组如下
INFO:root:[-1, -1, -1, -1, -1, -1, -1, 4]
INFO:root:********************************************
INFO:root:处理模式串的下标6
INFO:root:假设这个子串是好字符:ab
INFO:root:这个子串上一次出现的位置是:3
INFO:root:暂时设置这个下标，现在这个数组如下
INFO:root:[-1, -1, -1, -1, -1, -1, 3, 4]
INFO:root:因为这个子串的子串或许有更好的位置，现在数组如下
INFO:root:[-1, -1, -1, -1, -1, -1, 4, 4]
INFO:root:********************************************
INFO:root:处理模式串的下标5
INFO:root:假设这个子串是好字符:cab
INFO:root:这个子串上一次出现的位置是:2
INFO:root:暂时设置这个下标，现在这个数组如下
INFO:root:[-1, -1, -1, -1, -1, 2, 4, 4]
INFO:root:因为这个子串的子串或许有更好的位置，现在数组如下
INFO:root:[-1, -1, -1, -1, -1, 4, 4, 4]
INFO:root:********************************************
INFO:root:处理模式串的下标4
INFO:root:假设这个子串是好字符:bcab
INFO:root:这个子串上一次出现的位置是:-1
INFO:root:暂时设置这个下标，现在这个数组如下
INFO:root:[-1,

[-1, 4, 4, 4, 4, 4, 4, 4]

In [64]:
from IPython.core.debugger import set_trace

def BM_matching(t: str, p: str):
    """这个是BM搜索算法"""
    # set_trace()
    i=0 # 这个指向匹配串的头部
    j=len(p)-1 # 这个相当于匹配串的下标，跟i相加，就能得到目标串的小标
    m=len(t) # 这个是目标串的长度
    # 循环啦，退出条件当然是越界
    _good_char=get_good_char_move(p) # 这个是装好字符串的
    _is_have_good =False
    logging.info("BB搜索算法开始循环*****************************")
    while i+j < m :
        logging.info("目标串：{0}".format(t))
        logging.info("模式串：{0}".format(" "*i + p))
        logging.info("i={0},j={1}".format(i,j))
        # 这里开始判断。
        # 首先判断是否匹配
        logging.info("p[j]={0},t[i+j]={1}".format(p[j] , t[i+j] ))
        if p[j] == t[i+j] :
            # 如果相等，意味着这个位置是好字符
            _is_have_good=True
            # j就减少1位，进行下一个的匹配
            logging.info("这个字符匹配，j少一位,现在j：{0}".format(j))
            j = j -1 
            # 如果j 越界了，那么说明找到相应的字符了吧
            if j == 0 :
                # 返回就是这个i，这个基地址啦。
                logging.info("搜索完成，下标为{0}".format(i))
                return i
        else:
            # 如果不相等，意味着这个位置是坏字符。
            # 然后就要计算转移位置啦
            # 首先计算根据坏字符的转移位置，#
            # 首先取得坏字符
            _bad_char = t[i+j]
            logging.info("到这里表示不匹配，找到坏字符 {0}".format(_bad_char))
            # 看看这个坏字符在模式串中最右边出现的位置
            _bad_char_index = p.rfind(_bad_char) # 我这里直接用的rfind只是作为一个例子，
            _bad_char_move= j -_bad_char_index # 目标串坏字符对应的位置 -  坏字符在模式串中出现的位置
            logging.info("根据坏字符法则需要移动 {0}".format(_bad_char_move))
            # 如果有好字符，才适用好字符法则吧
            if _is_have_good:
                _good_char_move = _good_char[j]
                logging.info("根据好字符法则需要移动 {}".format(_good_char_move))
                # 总的位置是2种移动的最大值
                i = i + max(_good_char_move,_bad_char_move)
                logging.info("移动模式串位数 {0}".format(i))
            else:
                # 如果没有好字符，那么就直接用坏字符的法则移动啦
                i = i+_bad_char_move
                logging.info("移动模式串位数 {0}".format(i))
            # 然后j要初始化啦
            j=len(p)-1
            _is_have_good =False
            
        logging.info("*************************************")
            
    return -1 # 没有找到啦。

In [65]:
BM_matching("hello world","world")

INFO:root:处理模式串:world
INFO:root:生成初始数组:[-1, -1, -1, -1, -1]
INFO:root:处理模式串的下标4
INFO:root:假设这个子串是好字符:d
INFO:root:这个子串上一次出现的位置是:-1
INFO:root:暂时设置这个下标，现在这个数组如下
INFO:root:[-1, -1, -1, -1, -1]
INFO:root:因为这个子串的子串或许有更好的位置，现在数组如下
INFO:root:[-1, -1, -1, -1, -1]
INFO:root:********************************************
INFO:root:处理模式串的下标3
INFO:root:假设这个子串是好字符:ld
INFO:root:这个子串上一次出现的位置是:-1
INFO:root:暂时设置这个下标，现在这个数组如下
INFO:root:[-1, -1, -1, -1, -1]
INFO:root:因为这个子串的子串或许有更好的位置，现在数组如下
INFO:root:[-1, -1, -1, -1, -1]
INFO:root:********************************************
INFO:root:处理模式串的下标2
INFO:root:假设这个子串是好字符:rld
INFO:root:这个子串上一次出现的位置是:-1
INFO:root:暂时设置这个下标，现在这个数组如下
INFO:root:[-1, -1, -1, -1, -1]
INFO:root:因为这个子串的子串或许有更好的位置，现在数组如下
INFO:root:[-1, -1, -1, -1, -1]
INFO:root:********************************************
INFO:root:处理模式串的下标1
INFO:root:假设这个子串是好字符:orld
INFO:root:这个子串上一次出现的位置是:-1
INFO:root:暂时设置这个下标，现在这个数组如下
INFO:root:[-1, -1, -1, -1, -1]
INFO:root:因为这个子串的子串或许有更好的位置，现在数组如下
INFO:root:[-1, -1, -

6